In [2]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def firsts(list):
    answer=[]
    for pair in list:
        answer=answer+[pair[0]]
    return answer

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jLeoPowerConstantTerm(m,power,multiplier):
    trun=jLeo(power,m,multiplier)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def last_element(listo):return listo[len(listo)-1]
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

print("okay")

okay


In [ ]:
#output snipped; I interrupted this after k = 94.
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [1..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst= jpowerConstantTerm(m,power)
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-nszfunction(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
#below was originally done in a separate cell
#import pickle
#wfile = open('/Users/barrybrent/26oct22no1.txt','wb')
#t = pickle.dumps(str(polydata))
#wfile.write(t)
#wfile.close()
---------------------------------------------------------------------------------------------------
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [95..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst=QQ(jpowerConstantTerm(m,power))
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-broadhurst(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
import pickle
wfile = open('/Users/barrybrent/26oct22no2.txt','wb')
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()
print("tests:")
print(tests)

In [2]:
a=[1,2,3];b=[4,5,6];c=a+b;print(c)

[1, 2, 3, 4, 5, 6]


In [6]:
import pickle
rsfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)

rtfile = open('/Users/barrybrent/26oct22no2.txt','rb')
rt=pickle.load(rtfile)
rtfile.close()
t=stripQuotationMarks(rt)

u=s+t

wufile = open('/Users/barrybrent/5dec22no1.txt','wb')
u = pickle.dumps(str(u))
wufile.write(u)

1968604

In [8]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print(len(s))

100


In [94]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
mp=0
data=[]
p=2
tests=[]
tupls=[]
dk=1
for h in [0..len(s)-1]:
        k=s[h][0]
        if digitsum(p,k)==dk: 
            poly=D(s[h][1])
            for m in [3..4100]:
                if ZZ(mod(m,p))==mp: # mp = 0 => m is even
                    #dm=ZZ(m/p).digits(p)
                    dm=digitsum(p,m)
                    a=order(p,m)
                    mp2=mod(m,p^2) # p = 2 => mp2 = mod(m,4); m is even, so mp2 = 0 or 2.
                    if mp2==0: # so ord_2(m) >= 2
                            polym=evaluate(poly,m)
                            o=order(p,polym)
                            test=o-((a+2)*k+3)
                            tupl=(k,m,test)
                            tests=tests+[test]
                            print(tupl)
                            tupls=tupls+[tupl]
ut=uniteList(tests);print();print("tests:",ut)

(1, 4, 0)
(1, 8, 0)
(1, 12, 0)
(1, 16, 0)
(1, 20, 0)
(1, 24, 0)
(1, 28, 0)
(1, 32, 0)
(1, 36, 0)
(1, 40, 0)
(1, 44, 0)
(1, 48, 0)
(1, 52, 0)
(1, 56, 0)
(1, 60, 0)
(1, 64, 0)
(1, 68, 0)
(1, 72, 0)
(1, 76, 0)
(1, 80, 0)
(1, 84, 0)
(1, 88, 0)
(1, 92, 0)
(1, 96, 0)
(1, 100, 0)
(1, 104, 0)
(1, 108, 0)
(1, 112, 0)
(1, 116, 0)
(1, 120, 0)
(1, 124, 0)
(1, 128, 0)
(1, 132, 0)
(1, 136, 0)
(1, 140, 0)
(1, 144, 0)
(1, 148, 0)
(1, 152, 0)
(1, 156, 0)
(1, 160, 0)
(1, 164, 0)
(1, 168, 0)
(1, 172, 0)
(1, 176, 0)
(1, 180, 0)
(1, 184, 0)
(1, 188, 0)
(1, 192, 0)
(1, 196, 0)
(1, 200, 0)
(1, 204, 0)
(1, 208, 0)
(1, 212, 0)
(1, 216, 0)
(1, 220, 0)
(1, 224, 0)
(1, 228, 0)
(1, 232, 0)
(1, 236, 0)
(1, 240, 0)
(1, 244, 0)
(1, 248, 0)
(1, 252, 0)
(1, 256, 0)
(1, 260, 0)
(1, 264, 0)
(1, 268, 0)
(1, 272, 0)
(1, 276, 0)
(1, 280, 0)
(1, 284, 0)
(1, 288, 0)
(1, 292, 0)
(1, 296, 0)
(1, 300, 0)
(1, 304, 0)
(1, 308, 0)
(1, 312, 0)
(1, 316, 0)
(1, 320, 0)
(1, 324, 0)
(1, 328, 0)
(1, 332, 0)
(1, 336, 0)
(1, 340, 0)
(1, 34

(1, 3348, 0)
(1, 3352, 0)
(1, 3356, 0)
(1, 3360, 0)
(1, 3364, 0)
(1, 3368, 0)
(1, 3372, 0)
(1, 3376, 0)
(1, 3380, 0)
(1, 3384, 0)
(1, 3388, 0)
(1, 3392, 0)
(1, 3396, 0)
(1, 3400, 0)
(1, 3404, 0)
(1, 3408, 0)
(1, 3412, 0)
(1, 3416, 0)
(1, 3420, 0)
(1, 3424, 0)
(1, 3428, 0)
(1, 3432, 0)
(1, 3436, 0)
(1, 3440, 0)
(1, 3444, 0)
(1, 3448, 0)
(1, 3452, 0)
(1, 3456, 0)
(1, 3460, 0)
(1, 3464, 0)
(1, 3468, 0)
(1, 3472, 0)
(1, 3476, 0)
(1, 3480, 0)
(1, 3484, 0)
(1, 3488, 0)
(1, 3492, 0)
(1, 3496, 0)
(1, 3500, 0)
(1, 3504, 0)
(1, 3508, 0)
(1, 3512, 0)
(1, 3516, 0)
(1, 3520, 0)
(1, 3524, 0)
(1, 3528, 0)
(1, 3532, 0)
(1, 3536, 0)
(1, 3540, 0)
(1, 3544, 0)
(1, 3548, 0)
(1, 3552, 0)
(1, 3556, 0)
(1, 3560, 0)
(1, 3564, 0)
(1, 3568, 0)
(1, 3572, 0)
(1, 3576, 0)
(1, 3580, 0)
(1, 3584, 0)
(1, 3588, 0)
(1, 3592, 0)
(1, 3596, 0)
(1, 3600, 0)
(1, 3604, 0)
(1, 3608, 0)
(1, 3612, 0)
(1, 3616, 0)
(1, 3620, 0)
(1, 3624, 0)
(1, 3628, 0)
(1, 3632, 0)
(1, 3636, 0)
(1, 3640, 0)
(1, 3644, 0)
(1, 3648, 0)
(1, 3652, 0)

(2, 3428, 0)
(2, 3432, 0)
(2, 3436, 0)
(2, 3440, 0)
(2, 3444, 0)
(2, 3448, 0)
(2, 3452, 0)
(2, 3456, 0)
(2, 3460, 0)
(2, 3464, 0)
(2, 3468, 0)
(2, 3472, 0)
(2, 3476, 0)
(2, 3480, 0)
(2, 3484, 0)
(2, 3488, 0)
(2, 3492, 0)
(2, 3496, 0)
(2, 3500, 0)
(2, 3504, 0)
(2, 3508, 0)
(2, 3512, 0)
(2, 3516, 0)
(2, 3520, 0)
(2, 3524, 0)
(2, 3528, 0)
(2, 3532, 0)
(2, 3536, 0)
(2, 3540, 0)
(2, 3544, 0)
(2, 3548, 0)
(2, 3552, 0)
(2, 3556, 0)
(2, 3560, 0)
(2, 3564, 0)
(2, 3568, 0)
(2, 3572, 0)
(2, 3576, 0)
(2, 3580, 0)
(2, 3584, 0)
(2, 3588, 0)
(2, 3592, 0)
(2, 3596, 0)
(2, 3600, 0)
(2, 3604, 0)
(2, 3608, 0)
(2, 3612, 0)
(2, 3616, 0)
(2, 3620, 0)
(2, 3624, 0)
(2, 3628, 0)
(2, 3632, 0)
(2, 3636, 0)
(2, 3640, 0)
(2, 3644, 0)
(2, 3648, 0)
(2, 3652, 0)
(2, 3656, 0)
(2, 3660, 0)
(2, 3664, 0)
(2, 3668, 0)
(2, 3672, 0)
(2, 3676, 0)
(2, 3680, 0)
(2, 3684, 0)
(2, 3688, 0)
(2, 3692, 0)
(2, 3696, 0)
(2, 3700, 0)
(2, 3704, 0)
(2, 3708, 0)
(2, 3712, 0)
(2, 3716, 0)
(2, 3720, 0)
(2, 3724, 0)
(2, 3728, 0)
(2, 3732, 0)

(4, 2452, 0)
(4, 2456, 0)
(4, 2460, 0)
(4, 2464, 0)
(4, 2468, 0)
(4, 2472, 0)
(4, 2476, 0)
(4, 2480, 0)
(4, 2484, 0)
(4, 2488, 0)
(4, 2492, 0)
(4, 2496, 0)
(4, 2500, 0)
(4, 2504, 0)
(4, 2508, 0)
(4, 2512, 0)
(4, 2516, 0)
(4, 2520, 0)
(4, 2524, 0)
(4, 2528, 0)
(4, 2532, 0)
(4, 2536, 0)
(4, 2540, 0)
(4, 2544, 0)
(4, 2548, 0)
(4, 2552, 0)
(4, 2556, 0)
(4, 2560, 0)
(4, 2564, 0)
(4, 2568, 0)
(4, 2572, 0)
(4, 2576, 0)
(4, 2580, 0)
(4, 2584, 0)
(4, 2588, 0)
(4, 2592, 0)
(4, 2596, 0)
(4, 2600, 0)
(4, 2604, 0)
(4, 2608, 0)
(4, 2612, 0)
(4, 2616, 0)
(4, 2620, 0)
(4, 2624, 0)
(4, 2628, 0)
(4, 2632, 0)
(4, 2636, 0)
(4, 2640, 0)
(4, 2644, 0)
(4, 2648, 0)
(4, 2652, 0)
(4, 2656, 0)
(4, 2660, 0)
(4, 2664, 0)
(4, 2668, 0)
(4, 2672, 0)
(4, 2676, 0)
(4, 2680, 0)
(4, 2684, 0)
(4, 2688, 0)
(4, 2692, 0)
(4, 2696, 0)
(4, 2700, 0)
(4, 2704, 0)
(4, 2708, 0)
(4, 2712, 0)
(4, 2716, 0)
(4, 2720, 0)
(4, 2724, 0)
(4, 2728, 0)
(4, 2732, 0)
(4, 2736, 0)
(4, 2740, 0)
(4, 2744, 0)
(4, 2748, 0)
(4, 2752, 0)
(4, 2756, 0)

(8, 1960, 0)
(8, 1964, 0)
(8, 1968, 0)
(8, 1972, 0)
(8, 1976, 0)
(8, 1980, 0)
(8, 1984, 0)
(8, 1988, 0)
(8, 1992, 0)
(8, 1996, 0)
(8, 2000, 0)
(8, 2004, 0)
(8, 2008, 0)
(8, 2012, 0)
(8, 2016, 0)
(8, 2020, 0)
(8, 2024, 0)
(8, 2028, 0)
(8, 2032, 0)
(8, 2036, 0)
(8, 2040, 0)
(8, 2044, 0)
(8, 2048, 0)
(8, 2052, 0)
(8, 2056, 0)
(8, 2060, 0)
(8, 2064, 0)
(8, 2068, 0)
(8, 2072, 0)
(8, 2076, 0)
(8, 2080, 0)
(8, 2084, 0)
(8, 2088, 0)
(8, 2092, 0)
(8, 2096, 0)
(8, 2100, 0)
(8, 2104, 0)
(8, 2108, 0)
(8, 2112, 0)
(8, 2116, 0)
(8, 2120, 0)
(8, 2124, 0)
(8, 2128, 0)
(8, 2132, 0)
(8, 2136, 0)
(8, 2140, 0)
(8, 2144, 0)
(8, 2148, 0)
(8, 2152, 0)
(8, 2156, 0)
(8, 2160, 0)
(8, 2164, 0)
(8, 2168, 0)
(8, 2172, 0)
(8, 2176, 0)
(8, 2180, 0)
(8, 2184, 0)
(8, 2188, 0)
(8, 2192, 0)
(8, 2196, 0)
(8, 2200, 0)
(8, 2204, 0)
(8, 2208, 0)
(8, 2212, 0)
(8, 2216, 0)
(8, 2220, 0)
(8, 2224, 0)
(8, 2228, 0)
(8, 2232, 0)
(8, 2236, 0)
(8, 2240, 0)
(8, 2244, 0)
(8, 2248, 0)
(8, 2252, 0)
(8, 2256, 0)
(8, 2260, 0)
(8, 2264, 0)

(16, 696, 0)
(16, 700, 0)
(16, 704, 0)
(16, 708, 0)
(16, 712, 0)
(16, 716, 0)
(16, 720, 0)
(16, 724, 0)
(16, 728, 0)
(16, 732, 0)
(16, 736, 0)
(16, 740, 0)
(16, 744, 0)
(16, 748, 0)
(16, 752, 0)
(16, 756, 0)
(16, 760, 0)
(16, 764, 0)
(16, 768, 0)
(16, 772, 0)
(16, 776, 0)
(16, 780, 0)
(16, 784, 0)
(16, 788, 0)
(16, 792, 0)
(16, 796, 0)
(16, 800, 0)
(16, 804, 0)
(16, 808, 0)
(16, 812, 0)
(16, 816, 0)
(16, 820, 0)
(16, 824, 0)
(16, 828, 0)
(16, 832, 0)
(16, 836, 0)
(16, 840, 0)
(16, 844, 0)
(16, 848, 0)
(16, 852, 0)
(16, 856, 0)
(16, 860, 0)
(16, 864, 0)
(16, 868, 0)
(16, 872, 0)
(16, 876, 0)
(16, 880, 0)
(16, 884, 0)
(16, 888, 0)
(16, 892, 0)
(16, 896, 0)
(16, 900, 0)
(16, 904, 0)
(16, 908, 0)
(16, 912, 0)
(16, 916, 0)
(16, 920, 0)
(16, 924, 0)
(16, 928, 0)
(16, 932, 0)
(16, 936, 0)
(16, 940, 0)
(16, 944, 0)
(16, 948, 0)
(16, 952, 0)
(16, 956, 0)
(16, 960, 0)
(16, 964, 0)
(16, 968, 0)
(16, 972, 0)
(16, 976, 0)
(16, 980, 0)
(16, 984, 0)
(16, 988, 0)
(16, 992, 0)
(16, 996, 0)
(16, 1000, 0

(16, 3136, 0)
(16, 3140, 0)
(16, 3144, 0)
(16, 3148, 0)
(16, 3152, 0)
(16, 3156, 0)
(16, 3160, 0)
(16, 3164, 0)
(16, 3168, 0)
(16, 3172, 0)
(16, 3176, 0)
(16, 3180, 0)
(16, 3184, 0)
(16, 3188, 0)
(16, 3192, 0)
(16, 3196, 0)
(16, 3200, 0)
(16, 3204, 0)
(16, 3208, 0)
(16, 3212, 0)
(16, 3216, 0)
(16, 3220, 0)
(16, 3224, 0)
(16, 3228, 0)
(16, 3232, 0)
(16, 3236, 0)
(16, 3240, 0)
(16, 3244, 0)
(16, 3248, 0)
(16, 3252, 0)
(16, 3256, 0)
(16, 3260, 0)
(16, 3264, 0)
(16, 3268, 0)
(16, 3272, 0)
(16, 3276, 0)
(16, 3280, 0)
(16, 3284, 0)
(16, 3288, 0)
(16, 3292, 0)
(16, 3296, 0)
(16, 3300, 0)
(16, 3304, 0)
(16, 3308, 0)
(16, 3312, 0)
(16, 3316, 0)
(16, 3320, 0)
(16, 3324, 0)
(16, 3328, 0)
(16, 3332, 0)
(16, 3336, 0)
(16, 3340, 0)
(16, 3344, 0)
(16, 3348, 0)
(16, 3352, 0)
(16, 3356, 0)
(16, 3360, 0)
(16, 3364, 0)
(16, 3368, 0)
(16, 3372, 0)
(16, 3376, 0)
(16, 3380, 0)
(16, 3384, 0)
(16, 3388, 0)
(16, 3392, 0)
(16, 3396, 0)
(16, 3400, 0)
(16, 3404, 0)
(16, 3408, 0)
(16, 3412, 0)
(16, 3416, 0)
(16, 3

(32, 1520, 0)
(32, 1524, 0)
(32, 1528, 0)
(32, 1532, 0)
(32, 1536, 0)
(32, 1540, 0)
(32, 1544, 0)
(32, 1548, 0)
(32, 1552, 0)
(32, 1556, 0)
(32, 1560, 0)
(32, 1564, 0)
(32, 1568, 0)
(32, 1572, 0)
(32, 1576, 0)
(32, 1580, 0)
(32, 1584, 0)
(32, 1588, 0)
(32, 1592, 0)
(32, 1596, 0)
(32, 1600, 0)
(32, 1604, 0)
(32, 1608, 0)
(32, 1612, 0)
(32, 1616, 0)
(32, 1620, 0)
(32, 1624, 0)
(32, 1628, 0)
(32, 1632, 0)
(32, 1636, 0)
(32, 1640, 0)
(32, 1644, 0)
(32, 1648, 0)
(32, 1652, 0)
(32, 1656, 0)
(32, 1660, 0)
(32, 1664, 0)
(32, 1668, 0)
(32, 1672, 0)
(32, 1676, 0)
(32, 1680, 0)
(32, 1684, 0)
(32, 1688, 0)
(32, 1692, 0)
(32, 1696, 0)
(32, 1700, 0)
(32, 1704, 0)
(32, 1708, 0)
(32, 1712, 0)
(32, 1716, 0)
(32, 1720, 0)
(32, 1724, 0)
(32, 1728, 0)
(32, 1732, 0)
(32, 1736, 0)
(32, 1740, 0)
(32, 1744, 0)
(32, 1748, 0)
(32, 1752, 0)
(32, 1756, 0)
(32, 1760, 0)
(32, 1764, 0)
(32, 1768, 0)
(32, 1772, 0)
(32, 1776, 0)
(32, 1780, 0)
(32, 1784, 0)
(32, 1788, 0)
(32, 1792, 0)
(32, 1796, 0)
(32, 1800, 0)
(32, 1

(32, 4080, 0)
(32, 4084, 0)
(32, 4088, 0)
(32, 4092, 0)
(32, 4096, 0)
(32, 4100, 0)
(64, 4, 0)
(64, 8, 0)
(64, 12, 0)
(64, 16, 0)
(64, 20, 0)
(64, 24, 0)
(64, 28, 0)
(64, 32, 0)
(64, 36, 0)
(64, 40, 0)
(64, 44, 0)
(64, 48, 0)
(64, 52, 0)
(64, 56, 0)
(64, 60, 0)
(64, 64, 0)
(64, 68, 0)
(64, 72, 0)
(64, 76, 0)
(64, 80, 0)
(64, 84, 0)
(64, 88, 0)
(64, 92, 0)
(64, 96, 0)
(64, 100, 0)
(64, 104, 0)
(64, 108, 0)
(64, 112, 0)
(64, 116, 0)
(64, 120, 0)
(64, 124, 0)
(64, 128, 0)
(64, 132, 0)
(64, 136, 0)
(64, 140, 0)
(64, 144, 0)
(64, 148, 0)
(64, 152, 0)
(64, 156, 0)
(64, 160, 0)
(64, 164, 0)
(64, 168, 0)
(64, 172, 0)
(64, 176, 0)
(64, 180, 0)
(64, 184, 0)
(64, 188, 0)
(64, 192, 0)
(64, 196, 0)
(64, 200, 0)
(64, 204, 0)
(64, 208, 0)
(64, 212, 0)
(64, 216, 0)
(64, 220, 0)
(64, 224, 0)
(64, 228, 0)
(64, 232, 0)
(64, 236, 0)
(64, 240, 0)
(64, 244, 0)
(64, 248, 0)
(64, 252, 0)
(64, 256, 0)
(64, 260, 0)
(64, 264, 0)
(64, 268, 0)
(64, 272, 0)
(64, 276, 0)
(64, 280, 0)
(64, 284, 0)
(64, 288, 0)
(64, 2

(64, 2476, 0)
(64, 2480, 0)
(64, 2484, 0)
(64, 2488, 0)
(64, 2492, 0)
(64, 2496, 0)
(64, 2500, 0)
(64, 2504, 0)
(64, 2508, 0)
(64, 2512, 0)
(64, 2516, 0)
(64, 2520, 0)
(64, 2524, 0)
(64, 2528, 0)
(64, 2532, 0)
(64, 2536, 0)
(64, 2540, 0)
(64, 2544, 0)
(64, 2548, 0)
(64, 2552, 0)
(64, 2556, 0)
(64, 2560, 0)
(64, 2564, 0)
(64, 2568, 0)
(64, 2572, 0)
(64, 2576, 0)
(64, 2580, 0)
(64, 2584, 0)
(64, 2588, 0)
(64, 2592, 0)
(64, 2596, 0)
(64, 2600, 0)
(64, 2604, 0)
(64, 2608, 0)
(64, 2612, 0)
(64, 2616, 0)
(64, 2620, 0)
(64, 2624, 0)
(64, 2628, 0)
(64, 2632, 0)
(64, 2636, 0)
(64, 2640, 0)
(64, 2644, 0)
(64, 2648, 0)
(64, 2652, 0)
(64, 2656, 0)
(64, 2660, 0)
(64, 2664, 0)
(64, 2668, 0)
(64, 2672, 0)
(64, 2676, 0)
(64, 2680, 0)
(64, 2684, 0)
(64, 2688, 0)
(64, 2692, 0)
(64, 2696, 0)
(64, 2700, 0)
(64, 2704, 0)
(64, 2708, 0)
(64, 2712, 0)
(64, 2716, 0)
(64, 2720, 0)
(64, 2724, 0)
(64, 2728, 0)
(64, 2732, 0)
(64, 2736, 0)
(64, 2740, 0)
(64, 2744, 0)
(64, 2748, 0)
(64, 2752, 0)
(64, 2756, 0)
(64, 2